## Sensitive analysis:


In [9]:
from sast.sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd


### Select Datasets for hypertunning RSAST

It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 100 and 10000 for the selected dataset.

In [10]:
ds_sens = tsc_dataset_names.univariate_equal_length
#ds_sens = ['ACSF1', 'Adiac', 'ArrowHead', 'Beef', 'BeetleFly', 'BirdChicken', 'BME', 'Car', 'CBF', 'Chinatown', 'ChlorineConcentration', 'CinCECGTorso', 'Coffee']
ds_sens = ['Chinatown']
max_ds=len(ds_sens) #exploring dataset in UEA & UCR Time Series Classification Repository
#print(ds_sens)
print(ds_sens)


['Chinatown']


In [17]:
runs = 10
range_total = [1,10]
#list
df_result = {}
list_score = []
list_time = []
list_dataset = []
list_hyperparameter = []
list_method = []
list_rpoint = []
list_nb_per_class = []

for ds in ds_sens:
    print("ds="+ds)
    X_train, y_train = load_UCR_UEA_dataset(name=ds, extract_path='data', split="train", return_type="numpy2d")
    X_test, y_test = load_UCR_UEA_dataset(name=ds, extract_path='data', split="test", return_type="numpy2d")
    for i in range(runs):
        for p in range_total:
            for g in range_total:
                if p ==0:
                    p=1
                if g ==0:
                    g=1
                start = time.time()
                random_state = None
                rsast_ridge = RSAST(n_random_points=p,nb_inst_per_class=g, classifier=RidgeClassifierCV())
                rsast_ridge.fit(X_train, y_train)
                end = time.time()
                list_score.append(rsast_ridge.score(X_test,y_test))
                list_time.append(end-start)
                list_dataset.append(ds)
                list_hyperparameter.append("n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                list_rpoint.append(str(p))
                list_nb_per_class.append(str(g))
                list_method.append("Rsast")
                print("n_random_points="+str(p)+" nb_inst_per_class="+str(g))
df_result['accuracy']=list_score
df_result['time']=list_time
df_result['dataset_name']=list_dataset
df_result['classifier_name']=list_hyperparameter
df_result['rpoint']=list_rpoint
df_result['nb_per_class']=list_nb_per_class
df_result['method']=list_method
df_result=pd.DataFrame(df_result)

ds=Chinatown
n_random_points=1 nb_inst_per_class=1
n_random_points=1 nb_inst_per_class=10
n_random_points=10 nb_inst_per_class=1
n_random_points=10 nb_inst_per_class=10
n_random_points=1 nb_inst_per_class=1
n_random_points=1 nb_inst_per_class=10
n_random_points=10 nb_inst_per_class=1
n_random_points=10 nb_inst_per_class=10
n_random_points=1 nb_inst_per_class=1
n_random_points=1 nb_inst_per_class=10
n_random_points=10 nb_inst_per_class=1
n_random_points=10 nb_inst_per_class=10
n_random_points=1 nb_inst_per_class=1
n_random_points=1 nb_inst_per_class=10
n_random_points=10 nb_inst_per_class=1
n_random_points=10 nb_inst_per_class=10
n_random_points=1 nb_inst_per_class=1
n_random_points=1 nb_inst_per_class=10
n_random_points=10 nb_inst_per_class=1
n_random_points=10 nb_inst_per_class=10
n_random_points=1 nb_inst_per_class=1
n_random_points=1 nb_inst_per_class=10
n_random_points=10 nb_inst_per_class=1
n_random_points=10 nb_inst_per_class=10
n_random_points=1 nb_inst_per_class=1
n_random_poin

In [25]:
# export a overall dataset with results
df_result.to_csv("df_overall_tunning.csv") 

# create and export a pivot table with the mean of score
pivot_table = pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
pivot_table.to_csv("df_pivot_tunning_hyper.csv") 


In [26]:
# create a pivot table with the mean of score by hyperparameter
pd.pivot_table(df_result, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10
rpoint,,
1,0.827988,0.952478
10,0.928571,0.979009


In [27]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,1,10
rpoint,,
1,0.017251,0.000476
10,0.002260,0.000028


In [28]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60

nb_per_class,1,10
rpoint,,
1,0.000250,0.000832
10,0.000358,0.002030
